# 02 获取大体信息

In [1]:
import pandas as pd
from utils.get_main_info import get_info, _get_info_single_

DATA = "data/01-Name-Link.csv"
DATA_PATH = "data/02-Info.csv"

# 读取数据
name_link = pd.read_csv(DATA, encoding="utf-8")
name_link.head()

- You're loading a package from utils.


,name,link
0,重岳,https://prts.wiki/w/%E9%93%8E%E9%93%83
1,闪击,https://prts.wiki/w/%E6%B5%8A%E5%BF%83%E6%96%A...
2,伊芙利特,https://prts.wiki/w/%E8%80%80%E9%AA%91%E5%A3%A...
3,断崖,https://prts.wiki/w/%E4%BB%A4
4,炎客,https://prts.wiki/w/%E6%B7%B1%E6%B5%B7%E8%89%B2


In [2]:
# 将df拆解成几个列表
list_name = name_link['name'].tolist()
list_link = name_link['link'].tolist()

In [3]:
# table2 = _get_info_single_page("https://prts.wiki/w/%E9%98%BF%E7%B1%B3%E5%A8%85")
_get_info_single_("https://prts.wiki/w/%E9%98%BF%E7%B1%B3%E5%A8%85", verbose=True)

soup time: 1.0075325965881348
table time: 0.022555112838745117


['中坚术师',
 '攻击造成法术伤害',
 '70s',
 '18→20',
 '1',
 '1.6s',
 ['699', '958', '1198', '1480', '200'],
 ['276', '390', '514', '612', '70'],
 ['48', '81', '110', '121', '0'],
 ['10', '10', '15', '20', '0'],
 [9, 10, 10],
 '术师']

In [4]:
# # 用随机5个数据测试
# np.random.seed(42)
# test_index = np.random.randint(0, len(name_link), 5)

# # 用这5个数据测试
# test_data = name_link.iloc[test_index]
# print(test_data.iloc[:, 0])

# get_info(test_data)

In [5]:
info = get_info(name_link,verbose=True)

  0%|          | 0/300 [00:00<?, ?it/s]

current operator: 重岳


  0%|          | 1/300 [00:00<02:06,  2.37it/s]

current operator: 闪击


  1%|          | 2/300 [00:01<04:14,  1.17it/s]

...

 99%|█████████▉| 298/300 [03:31<00:03,  1.52s/it]

current operator: 拜松


100%|█████████▉| 299/300 [03:31<00:01,  1.23s/it]

current operator: 迷迭香


100%|██████████| 300/300 [03:32<00:00,  1.41it/s]


In [6]:
# 在info的开头加入name_link中的name列
info = pd.concat([name_link.iloc[:,0], info], axis=1)

# 将name列改名为干员名字列
info.rename(columns={"name": "干员名字"}, inplace=True)

In [7]:
# 将info存入文件
info.to_csv(DATA_PATH, index=False)